In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread

folder = 'train/'
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    filename = folder + 'dog.' + str(i) + '.jpg'
    image = imread(filename)
    pyplot.imshow(image)

pyplot.show()

In [ ]:
from matplotlib import pyplot
from matplotlib.image import imread

folder = 'train/'
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    filename = folder + 'cat.' + str(i) + '.jpg'
    image = imread(filename)
    pyplot.imshow(image)

pyplot.show()

In [ ]:
from os import listdir
from numpy import asarray
from numpy import save
from keras_preprocessing.image import load_img
from keras_preprocessing.image import img_to_array

folder = 'train/'
photos, labels = list(), list()
for file in listdir(folder):
    output = 0.0
    if file.startswith('cat'):
        output = 1.0
    photo = load_img(folder + file, target_size=(200, 200))
    photo = img_to_array(photo)
    photos.append(photo)
    labels.append(output)

photos = asarray(photos)
labels = asarray(labels)
print(photos.shape, labels.shape)
save('dogs_vs_cats_photos.npy', photos)
save('dogs_vs_cats_labels.npy', labels)

: 